## QuestGen-LLM: Fine-Tuning

This notebook covers the fine-tuning of various pre-trained _large language models_ (LLMs) on the prepared ["quest"](../data/quests_train.json) dataset. Each language model applied is trained and validated on the dataset (with frozen parameters) and the results of these evaluations are compared. The LLMs employed for this application are listed in the following table with their respective parameter count.

| S. No. | Large Language Model             | Parameters | Developed By | Notes                                                 |
| :----: | :------------------------------- | :--------: | :----------: | :---------------------------------------------------- |
|   1.   | GPT-2[^1]                        |    124M    |    OpenAI    | Base model from the GPT-2 family                      |
|   2.   | GPT-2 Medium[^2]                 |    355M    |    OpenAI    | Larger variant with improved language modeling        |
|   3.   | GPT-2 Large[^3]                  |    774M    |    OpenAI    | Capable of generating more coherent longer text       |
|   4.   | Llama-3.2-1B-Instruct[^4] †      |     1B     |     Meta     | Instruction-tuned model for question-answering        |
|   5.   | TinyLlama-1.1B-Chat-v1.0[^5] \*† |    1.1B    |  TinyLlama   | Lightweight chat-tuned model for constrained hardware |

> Fine-tuning uses _supervised fine-tuning_\* (SHF) and _reinforcement learning with human feedback_† (RLHF).

<!-- References -->

[^1]: https://huggingface.co/openai-community/gpt2
[^2]: https://huggingface.co/openai-community/gpt2-medium
[^3]: https://huggingface.co/openai-community/gpt2-large
[^4]: https://huggingface.co/meta-llama/Llama-3.2-1B-Instruct
[^5]: https://huggingface.co/TinyLlama/TinyLlama-1.1B-Chat-v1.0


In [1]:
from __future__ import annotations

import json
import os
import shutil
import sys
import time
from dataclasses import dataclass, field
from os import PathLike
from pathlib import Path
from typing import Any, Final, Optional

In [2]:
import torch
from datasets import Dataset, DatasetDict, load_dataset
from huggingface_hub import login
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    DataCollatorForLanguageModeling,
    EarlyStoppingCallback,
    PreTrainedModel,
    PreTrainedTokenizerFast,
    PreTrainedTokenizer,
    Trainer,
    TrainerCallback,
    TrainerControl,
    TrainerState,
    TrainingArguments,
    set_seed,
)
from transformers.integrations import TensorBoardCallback
from transformers.tokenization_utils_base import BatchEncoding

In [3]:
root: str = str(Path.cwd().parent.resolve())
if root not in sys.path:
    sys.path.insert(0, root)

In [4]:
from utils.dirpath import get_cache_dirpath, get_target_dirpath

In [5]:
# Get the HF access token from the environment
HF_ACCESS_TOKEN: Final[str] = os.getenv("HUGGINGFACE_HUB_TOKEN")

# Save the HF token to ~/.huggingface/token
login(token=HF_ACCESS_TOKEN)

In [6]:
# Map for the model identifiers: (model_key -> model_id)
MODEL_IDENTIFIERS: Final[dict[str, str]] = {
    "gpt2": "openai-community/gpt2",
    "gpt2-medium": "openai-community/gpt2-medium",
    "gpt2-large": "openai-community/gpt2-large",
    "llama-3.2-1b-instruct": "meta-llama/Llama-3.2-1B-Instruct",
    "tinyllama-1.1b-chat": "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
}

In [7]:
data_dir: Path = get_target_dirpath("data")

# Load the quest dataset
quest_set: DatasetDict = load_dataset(
    "text",
    data_files={
        "train": str(data_dir / "quests_train.txt"),
        "val": str(data_dir / "quests_val.txt"),
        "test": str(data_dir / "quests_test.txt"),
    },
    cache_dir=str(data_dir / ".cache"),
)
quest_set

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 19954
    })
    val: Dataset({
        features: ['text'],
        num_rows: 2486
    })
    test: Dataset({
        features: ['text'],
        num_rows: 2394
    })
})

In [8]:
quest_set["train"][:21]

{'text': ['### Instruction:',
  'Generate a video game quest description based on the following structured information.',
  '',
  '### Input:',
  'Quest Name: Perilous Passage',
  'Objective: save the Mana Queen',
  'First Tasks: go through the gate to the Forsaken Vaults',
  'First Task Locations: Forsaken Vaults - a perilous dungeon',
  'Quest Giver: NONE - NONE (location: NONE)',
  'Reward: NONE -  (amount: 1)',
  'Characters: Mana Queen - a good female spirit (location: Forsaken Vaults)',
  'Tools: NONE',
  'Locations: NONE',
  'Items: NONE',
  'Enemies: NONE',
  'Groups: NONE',
  'Title: Torchlight II',
  'Motivation: NONE',
  '',
  '### Response:',
  "The Mana Queen has come and gone Through this gate, she journeyed on. Follow her and pay the cost. Hasten forth, or she'll be lost."]}

In [19]:
quest_set["val"][22:43]

{'text': ['### Instruction:',
  'Generate a video game quest description based on the following structured information.',
  '',
  '### Input:',
  'Quest Name: A Child in the Lighthouse',
  "Objective: save Ardrouine's little son from worgs",
  'First Tasks: go to the abandoned lighthouse',
  'First Task Locations:  - abandoned lighthouse to the northwest',
  'Quest Giver: NONE - NONE (location: NONE)',
  'Reward:  - coins (amount: 60)',
  'Characters: NONE',
  'Tools: NONE',
  'Locations: NONE',
  'Items: NONE',
  'Enemies: NONE',
  'Groups: NONE',
  "Title: Baldur's Gate",
  'Motivation: NONE',
  '',
  '### Response:',
  "Please help me, I am just poor Ardrouine! I don't know where else to turn. My little boy was playing in that abandoned lighthouse to the northwest when a pack of worgs surrounded it. Please just turn them back, and I can coax him down. There's not much time! I can pay you 60 coins: this money is all my husband brought back from market this past week. My son's life is

In [10]:
# Map for the target modules: (model_key -> target_modules)
TARGET_MODULES: Final[dict[str, list[str]]] = {
    "gpt2": ["c_attn", "c_proj", "c_fc"],
    "gpt2-medium": ["c_attn", "c_proj", "c_fc"],
    "gpt2-large": ["c_attn", "c_proj", "c_fc"],
    "llama-3.2-1b-instruct": [
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    "tinyllama-1.1b-chat": [
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
}

In [11]:
# Set the constants for model tuning here
BATCH_SIZE: Final[int] = 4
N_EPOCHS: Final[int] = 1
SEED: Final[int] = 42
LR_RATE: Final[float] = 5e-7

MAX_LENGTH: Final[int] = 64
MAX_GRAD_NORM: Final[float] = 1.0
LOGGING_STEPS: Final[int] = 20
EVAL_STEPS: Final[int] = 50
WARMUP_STEPS: Final[int] = 500

SAVE_TOTAL_LIMIT: Final[int] = 1
EVAL_ACCUMULATION_STEPS: Final[int] = 2
GRADIENT_ACCUMULATION_STEPS: Final[int] = 2

GRADIENT_CHECKPOINTING: Final[bool] = False
LOAD_BEST_MODEL_AT_END: Final[bool] = True

ACTIVATE_FP16: Final[bool] = False
ACTIVATE_EVAL: Final[bool] = True
ACTIVATE_SAVE: Final[bool] = True
ACTIVATE_TEST: Final[bool] = True
ACTIVATE_LOGS: Final[bool] = False
ACTIVATE_TENSORBOARD: Final[bool] = True
ACTIVATE_CALLBACKS: Final[bool] = True
ACTIVATE_FULL: Final[bool] = False  # Full dataset or subset

FRACTION: Final[float] = 0.01  # 1% of the Quest dataset

In [12]:
# Prepare a subset of the quest dataset
quest_subset: DatasetDict = DatasetDict(
    {
        "train": quest_set["train"]
        .shuffle(seed=SEED)
        .select(range(int(FRACTION * quest_set["train"].num_rows))),
        "val": quest_set["val"]
        .shuffle(seed=SEED)
        .select(range(int(FRACTION * quest_set["val"].num_rows))),
        "test": quest_set["test"]
        .shuffle(seed=SEED)
        .select(range(int(FRACTION * quest_set["test"].num_rows))),
    }
)

In [13]:
@dataclass
class TrainingMetrics:
    train_losses: list[float] = field(default_factory=list)
    eval_losses: list[float] = field(default_factory=list)
    eval_results: list[dict[str, float]] = field(default_factory=list)


# Map for storing training metrics: (model_key -> metrics)
TRAINING_METRICS: dict[str, Optional[TrainingMetrics]] = {
    k: None for k in MODEL_IDENTIFIERS.keys()
}

In [14]:
class LossLoggerCallback(TrainerCallback):
    def __init__(self, metrics: TrainingMetrics):
        self.metrics: TrainingMetrics = metrics

    def on_log(
        self,
        args: TrainingArguments,
        state: TrainerState,
        control: TrainerControl,
        logs: Optional[dict[str, float]] = None,
        **kwargs: Any,
    ) -> None:
        if logs is None:
            return

        # Capture training losses during logging
        if "loss" in logs:
            self.metrics.train_losses.append(logs["loss"])

        # Capture evaluation losses during logging
        if "eval_loss" in logs:
            self.metrics.eval_losses.append(logs["eval_loss"])

        return super().on_log(args, state, control, **kwargs)

    def on_evaluate(
        self,
        args: TrainingArguments,
        state: TrainerState,
        control: TrainerControl,
        metrics: Optional[dict[str, float]] = None,
        **kwargs: Any,
    ) -> None:
        if metrics is None:
            return

        # Capture evaluation results on evaluation
        self.metrics.eval_results.append(metrics)

        return super().on_evaluate(args, state, control, **kwargs)

In [15]:
class QuestGenLLM:
    def __init__(
        self,
        tokenizer: PreTrainedTokenizer | PreTrainedTokenizerFast,
        model: PreTrainedModel,
        model_key: str,  # Alias for the model, e.g, "gpt2"
        model_id: str,  # Hugging Face model name, e.g., "openai-community/gpt2"
        fp16_available: bool,  # Mixed precision
        device: Optional[str] = None,
        dtype: Optional[str] = None,
        metrics: Optional[TrainingMetrics] = None,
    ):
        self.tokenizer: PreTrainedTokenizer | PreTrainedTokenizerFast = tokenizer
        self.model: PreTrainedModel = model
        self.model_key: str = model_key
        self.model_id: str = model_id
        self.fp16_available: bool = fp16_available

        # Automatically determine the device used by the model
        self.device: str = (
            device
            if isinstance(device, str)
            else str(getattr(self.model, "device", "N/A"))
        )

        # Automatically determine the dtype used by the model
        self.dtype: str = (
            dtype
            if isinstance(dtype, str)
            else str(getattr(self.model, "dtype", "N/A")).replace("torch.", "")
        )

        # Initialize dataclass for storing training metrics
        self.metrics: TrainingMetrics = (
            metrics if isinstance(metrics, TrainingMetrics) else TrainingMetrics()
        )

    @classmethod
    def from_pretrained(
        cls,
        model_key: str,
        model_id: str,
        cache_dir: PathLike = get_cache_dirpath("models"),
        seed: int = SEED,
        use_cpu: bool = False,
    ) -> QuestGenLLM:
        def apply_lora_adapter(
            model: PreTrainedModel,
            r: int = 8,
            alpha: int = 16,
            dropout: float = 0.1,
            task_type: str = "CAUSAL_LM",
        ) -> PreTrainedModel:
            # Prepare model for k-bit training
            model = prepare_model_for_kbit_training(model)

            # Set correct `fan_in_fan_out` based on model type
            #
            # False [default] - for Linear layers
            # True - for Conv1D layers (like GPT)
            fan_in_fan_out: bool = False
            if "gpt" in getattr(model.config, "model_type", "").lower():
                fan_in_fan_out = True

            # Define the LoRA config
            lora_config: LoraConfig = LoraConfig(
                r=r,
                lora_alpha=alpha,
                lora_dropout=dropout,
                target_modules=TARGET_MODULES[model_key],
                bias="none",
                task_type=task_type,
                fan_in_fan_out=fan_in_fan_out,
            )

            try:
                # Apply LoRA adapters to the model
                model = get_peft_model(model, lora_config)
            except Exception as e:
                print(f"[LoRAINFO] Adapter failed to apply: {e}")
                raise

            # Display information about the model parameters
            trainable_params: int = sum(
                p.numel() for p in model.parameters() if p.requires_grad
            )
            all_params: int = sum(p.numel() for p in model.parameters())
            trainable_percent: float = 100 * trainable_params / all_params
            print(
                "[LoRAINFO] trainable params: {:,} || all params: {:,} || trainable%: {:.4f}".format(
                    trainable_params, all_params, trainable_percent
                )
            )

            return model

        print(f"[DOWNLOAD] {model_key} ({model_id})")
        start_time: float = time.time()

        # Clear PyTorch's CUDA memory cache
        torch.cuda.empty_cache()

        # Set the random seed for reproducibility
        set_seed(seed)

        # Determine if mixed precision is available
        fp16_available: bool = (
            torch.cuda.is_available()
            and torch.cuda.get_device_capability(0)[0] >= 7
            and torch.cuda.get_device_capability(0)[1] >= 0
        )

        # Download the tokenizer using the model id
        tokenizer: PreTrainedTokenizerFast = AutoTokenizer.from_pretrained(
            model_id,
            cache_dir=(cache_dir / model_key),
            use_fast=True,
            token=HF_ACCESS_TOKEN,
            trust_remote_code=True,
        )

        model: PreTrainedModel
        if fp16_available and not use_cpu:
            # Set the bitsandbytes configuration for quantization
            bnb_config: BitsAndBytesConfig = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_use_double_quant=True,
                bnb_4bit_compute_dtype=torch.float16,
                # llm_int8_enable_fp32_cpu_offload=True,
            )

            # Download the model using the model id (for GPU)
            model = AutoModelForCausalLM.from_pretrained(
                model_id,
                torch_dtype=torch.float16,
                quantization_config=bnb_config,
                cache_dir=(cache_dir / model_key),
                token=HF_ACCESS_TOKEN,
                trust_remote_code=True,
                low_cpu_mem_usage=True,
            )
            model.to("cuda")
        else:
            # Download the model using the model id (for CPU)
            model = AutoModelForCausalLM.from_pretrained(
                model_id,
                torch_dtype=torch.float32,
                cache_dir=(cache_dir / model_key),
                token=HF_ACCESS_TOKEN,
                trust_remote_code=True,
                low_cpu_mem_usage=True,
            )
            model.to("cpu")

        # Apply the LoRA adapters to the model
        model = apply_lora_adapter(model)

        end_time: float = time.time()
        elapsed: float = end_time - start_time
        print(f'[COMPLETE] "{model_key}" ready in {elapsed:.2f}s.\n')

        return cls(tokenizer, model, model_key, model_id, fp16_available)

    def train_and_evaluate(
        self,
        dataset: DatasetDict = quest_set if ACTIVATE_FULL else quest_subset,
        max_length: int = MAX_LENGTH,
        learning_rate: int = LR_RATE,
        batch_size: int = BATCH_SIZE,
        epochs: int = N_EPOCHS,
        seed: int = SEED,
        max_grad_norm: float = MAX_GRAD_NORM,
        logging_steps: int = LOGGING_STEPS,
        eval_steps: int = EVAL_STEPS,
        warmup_steps: int = WARMUP_STEPS,
        gradient_checkpointing: bool = GRADIENT_CHECKPOINTING,
        load_best_model_at_end: bool = LOAD_BEST_MODEL_AT_END,
        save_total_limit: int = SAVE_TOTAL_LIMIT,
        eval_accumulation_steps: int = EVAL_ACCUMULATION_STEPS,
        gradient_accumulation_steps: int = GRADIENT_ACCUMULATION_STEPS,
        callbacks: list[TrainerCallback] = [
            EarlyStoppingCallback(early_stopping_patience=2) if ACTIVATE_EVAL else None,
            TensorBoardCallback() if ACTIVATE_TENSORBOARD else None,
        ],
        activate_fp16: bool = ACTIVATE_FP16,
        activate_eval: bool = ACTIVATE_EVAL,
        activate_save: bool = ACTIVATE_SAVE,
        activate_test: bool = ACTIVATE_TEST,
        activate_logs: bool = ACTIVATE_LOGS,
        activate_tensorboard: bool = ACTIVATE_TENSORBOARD,
        activate_callbacks: bool = ACTIVATE_CALLBACKS,
        output_dir: PathLike = get_target_dirpath("out"),
        logging_dir: PathLike = get_target_dirpath("logs"),
    ) -> TrainingMetrics:
        # Ensure the training and validation sets
        if not all(split in dataset for split in ["train", "val"]):
            raise ValueError("DatasetDict must contain both 'train' and 'val' splits.")

        # Ensure the output and logging directories
        os.makedirs(output_dir, exist_ok=True)
        os.makedirs(logging_dir, exist_ok=True)

        start_time: float
        end_time: float
        elapsed: float

        # Set the random seed for reproducibility
        set_seed(seed)

        # Set the padding token for the tokenizer
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        self.tokenizer.padding_side = "right"

        # Tokenize the dataset with `max_length` padding
        print(f"[TOKENIZE] {self.model_key} ({self.model_id})")
        start_time = time.time()
        tokenized_data: Dataset = dataset.map(
            QuestGenLLM.tokenize_dataset,
            batched=True,
            remove_columns=["text"],
            fn_kwargs={"tokenizer": self.tokenizer, "max_length": max_length},
        )
        end_time = time.time()
        elapsed = end_time - start_time
        print(f"[COMPLETE] Elapsed: {elapsed:.2f}s\n")

        # Set the model padding token (from the tokenizer)
        self.model.config.pad_token_id = self.tokenizer.pad_token_id

        # Turn off `use_cache` if `gradient_checkpointing` is on
        self.model.config.use_cache = not gradient_checkpointing

        # Set up the training configurations
        training_args: TrainingArguments = TrainingArguments(
            output_dir=(output_dir / self.model_key),
            learning_rate=learning_rate,
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
            num_train_epochs=epochs,
            log_level=("info" if activate_logs else "error"),
            logging_steps=logging_steps,
            eval_steps=eval_steps,
            eval_strategy=("epoch" if activate_eval else "no"),
            save_strategy=("epoch" if activate_save else "no"),
            logging_dir=(logging_dir / self.model_key),
            save_total_limit=save_total_limit,
            eval_accumulation_steps=eval_accumulation_steps,
            gradient_accumulation_steps=gradient_accumulation_steps,
            gradient_checkpointing=gradient_checkpointing,
            fp16=(self.fp16_available and activate_fp16),
            load_best_model_at_end=load_best_model_at_end,
            metric_for_best_model="eval_loss",
            seed=seed,
            report_to=("tensorboard" if activate_tensorboard else "none"),
            label_names=["labels"],
            max_grad_norm=max_grad_norm,
            warmup_steps=warmup_steps,
            logging_nan_inf_filter=True,
            skip_memory_metrics=True,
        )

        # Set up the data collator for the model
        data_collator: DataCollatorForLanguageModeling = (
            DataCollatorForLanguageModeling(tokenizer=self.tokenizer, mlm=False)
        )

        # Set up the callbacks for the trainer
        trainer_callbacks: list[TrainerCallback] = list(
            filter(lambda callback: callback is not None, callbacks)
        )
        if activate_callbacks:
            trainer_callbacks.append(LossLoggerCallback(self.metrics))

        # Prepare and run the trainer
        trainer: Trainer = Trainer(
            model=self.model,
            args=training_args,
            data_collator=data_collator,
            train_dataset=tokenized_data["train"],
            eval_dataset=(tokenized_data["val"] if activate_eval else None),
            callbacks=trainer_callbacks,
        )

        print(f"[FINETUNE] {self.model_key} ({self.model_id})")
        start_time = time.time()
        trainer.train()
        end_time = time.time()
        elapsed = end_time - start_time
        print(f"[COMPLETE] Elapsed: {elapsed:.2f}s\n")

        # Evaluate the model with the test set
        if activate_test:
            print(f"[EVALUATE] {self.model_key} ({self.model_id})")
            start_time = time.time()
            trainer.evaluate(eval_dataset=tokenized_data["test"])
            end_time = time.time()
            elapsed = end_time - start_time
            print(f"[COMPLETE] Elapsed: {elapsed:.2f}s")

        # Save the model and tokenizer for later use
        if activate_save:
            trainer.save_model()
            self.tokenizer.save_pretrained(save_directory=training_args.output_dir)

        # Add to the training metrics map
        TRAINING_METRICS[self.model_key] = self.metrics

        return self.metrics

    @staticmethod
    def tokenize_dataset(
        examples: dict[str, list[str]],
        tokenizer: PreTrainedTokenizer | PreTrainedTokenizerFast,
        max_length: int = MAX_LENGTH,
    ) -> dict[str, list[list[int]]]:
        encodings: BatchEncoding = tokenizer(
            examples["text"],
            padding="longest",
            truncation=True,
            max_length=max_length,
            return_tensors="pt",
        )

        input_ids: list[list[int]] = encodings["input_ids"]
        attention_mask: list[list[int]] = encodings["attention_mask"]

        labels: list[list[int]] = input_ids.clone()
        labels[input_ids == tokenizer.pad_token_id] = -100

        return {
            "input_ids": input_ids.tolist(),
            "attention_mask": attention_mask.tolist(),
            "labels": labels.tolist(),
        }

    def to_dict(self) -> dict[str, Any]:
        return {
            "model_key": self.model_key,
            "model_id": self.model_id,
            "device": self.device,
            "dtype": self.dtype,
            "vocab_size": getattr(self.tokenizer, "vocab_size", "unknown"),
            "max_length": getattr(self.tokenizer, "model_max_length", "unknown"),
            "model_type": getattr(
                getattr(self.model, "config", None), "model_type", "unknown"
            ),
            "num_parameters": self.model.num_parameters()
            if hasattr(self.model, "num_parameters")
            else "N/A",
            "fp16_available": self.fp16_available,
        }

    def clear_cache(self, cache_dir: PathLike = get_cache_dirpath("models")) -> None:
        def remove_dir(dir_path: PathLike) -> None:
            if os.path.exists(dir_path):
                shutil.rmtree(dir_path)
                print(f"Cache directory '{dir_path}' removed.")
            else:
                print(f"No cache directory found at '{dir_path}'.")

        remove_dir(cache_dir / self.model_key)

    def print_model_information(self) -> None:
        print(json.dumps(self.to_dict(), indent=2))

    def inspect(self) -> None:
        print(f"Tokenizer ({self.model_id}):\n{self.tokenizer}\n")
        print(f"Model ({self.model_id}):\n{self.model}\n")
        print(f"Configuration ({self.model_id}):\n{self.model.config}")
        print(f"Padding Token [PAD]             : {self.tokenizer.pad_token}")
        print(f"Begging of Sentence Token [BOS] : {self.tokenizer.bos_token}")
        print(f"End of Sentence Token [EOS]     : {self.tokenizer.eos_token}")
        print(f"Unknown Token [UNK]             : {self.tokenizer.unk_token}")
        print(f"Padding Side                    : {self.tokenizer.padding_side}")
        print(f"Padding Token ID                : {self.tokenizer.pad_token_id}\n")

    def __str__(self) -> str:
        return f"{self.model_key} ({self.model_id})"

In [16]:
# Build and train the GPT-2 Base model with the quest data
gpt2_base: QuestGenLLM = QuestGenLLM.from_pretrained(
    model_key="gpt2", model_id=MODEL_IDENTIFIERS["gpt2"]
)
gpt2_base.inspect()
gpt2_base.train_and_evaluate()

[DOWNLOAD] gpt2 (openai-community/gpt2)
[LoRAINFO] trainable params: 1,179,648 || all params: 125,619,456 || trainable%: 0.9391
[COMPLETE] "gpt2" ready in 1.25s.

Tokenizer (openai-community/gpt2):
GPT2TokenizerFast(name_or_path='openai-community/gpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}
)

Model (openai-community/gpt2):
PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), 

Epoch,Training Loss,Validation Loss
1,4.726000,4.960940


[COMPLETE] Elapsed: 14.99s

[EVALUATE] gpt2 (openai-community/gpt2)


[COMPLETE] Elapsed: 0.54s


TrainingMetrics(train_losses=[4.726], eval_losses=[4.960939884185791, 5.107525825500488], eval_results=[{'eval_loss': 4.960939884185791}, {'eval_loss': 5.107525825500488, 'eval_runtime': 0.5393, 'eval_samples_per_second': 42.648, 'eval_steps_per_second': 11.126, 'epoch': 1.0}])

In [17]:
# Build and train the GPT-2 Medium model with the quest data
gpt2_medium: QuestGenLLM = QuestGenLLM.from_pretrained(
    model_key="gpt2-medium", model_id=MODEL_IDENTIFIERS["gpt2-medium"]
)
gpt2_medium.inspect()
gpt2_medium.train_and_evaluate()

[DOWNLOAD] gpt2-medium (openai-community/gpt2-medium)
[LoRAINFO] trainable params: 3,145,728 || all params: 357,968,896 || trainable%: 0.8788
[COMPLETE] "gpt2-medium" ready in 1.38s.

Tokenizer (openai-community/gpt2-medium):
GPT2TokenizerFast(name_or_path='openai-community/gpt2-medium', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}
)

Model (openai-community/gpt2-medium):
PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 1024)
        (wpe): Embedding(1024, 1024)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-23): 24 x GPT2B

TrainingMetrics(train_losses=[4.2756], eval_losses=[4.421889781951904, 4.6748857498168945], eval_results=[{'eval_loss': 4.421889781951904, 'eval_runtime': 1.1975, 'eval_samples_per_second': 20.042, 'eval_steps_per_second': 5.01, 'epoch': 1.0}, {'eval_loss': 4.6748857498168945, 'eval_runtime': 1.7414, 'eval_samples_per_second': 13.207, 'eval_steps_per_second': 3.445, 'epoch': 1.0}])

In [ ]:
# Build and train the GPT-2 Large model with the quest data
gpt2_large: QuestGenLLM = QuestGenLLM.from_pretrained(
    model_key="gpt2-large", model_id=MODEL_IDENTIFIERS["gpt2-large"]
)
gpt2_large.inspect()
gpt2_large.train_and_evaluate()

In [ ]:
# Build and train the Llama 3.2 model with the quest data
llama32: QuestGenLLM = QuestGenLLM.from_pretrained(
    model_key="llama-3.2-1b-instruct",
    model_id=MODEL_IDENTIFIERS["llama-3.2-1b-instruct"],
)
llama32.inspect()
llama32.train_and_evaluate()

In [ ]:
# Build and train the TinyLlama model with the quest data
tinyllama: QuestGenLLM = QuestGenLLM.from_pretrained(
    model_key="tinyllama-1.1b-chat", model_id=MODEL_IDENTIFIERS["tinyllama-1.1b-chat"]
)
tinyllama.inspect()
tinyllama.train_and_evaluate()